In [15]:
from dotenv import load_dotenv
import requests
import os
from bs4 import BeautifulSoup

In [12]:
load_dotenv()

# Now you can access the API key like this:
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

In [13]:
#def get_data():
    #url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=" + api_key
    #response = requests.get(url)
    #values = response.json()
    #print(values)
    #return values
#get_data()

In [16]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

# Set up WebDriver
driver = webdriver.Chrome()

# Open FT.com search page
url = "https://www.ft.com/search?q=bitcoin&sort=relevance&isFirstView=true&dateRange=now-24h"
driver.get(url)

# Wait for the articles to load (up to 10 seconds)
try:
    articles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "o-teaser"))
    )
    print(f"✅ Found {len(articles)} articles!")
except:
    print("⚠️ No articles found. The page may not have loaded correctly.")

# Highlight the elements in the browser
for article in articles:
    driver.execute_script("arguments[0].style.border='3px solid red'", article)

# Keep browser open for review
input("Press Enter to close...")
driver.quit()



✅ Found 25 articles!


In [17]:
print(articles)

[<selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8c3", element="f.675C3EAEC7537EDF040AAA6EBE7770F9.d.C3493A8192E1230C804BA2DD9336E17A.e.92")>, <selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8c3", element="f.675C3EAEC7537EDF040AAA6EBE7770F9.d.C3493A8192E1230C804BA2DD9336E17A.e.93")>, <selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8c3", element="f.675C3EAEC7537EDF040AAA6EBE7770F9.d.C3493A8192E1230C804BA2DD9336E17A.e.94")>, <selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8c3", element="f.675C3EAEC7537EDF040AAA6EBE7770F9.d.C3493A8192E1230C804BA2DD9336E17A.e.95")>, <selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8c3", element="f.675C3EAEC7537EDF040AAA6EBE7770F9.d.C3493A8192E1230C804BA2DD9336E17A.e.96")>, <selenium.webdriver.remote.webelement.WebElement (session="b32d2821c1ff6d133777da0546e2b8